In [1]:
import os
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.asymmetric import dh

# Exercício 1

#### Neste exercício é pedido que se crie uma comunicação privada assíncrona entre um agente Emitter e um agente Receiver que cubra alguns aspetos enunciados a seguir.

Começamos por desenvolver uma função para derivar uma chave, baseada no funcionamento do protocolo DH para geração de chaves privadas e públicas. Para este protocolo, é necessário gerar parâmetros que devem ser os mesmos para os dois agentes e podem ser reutilizados.<br>
Inicialmente, gera-se a chave privada e depois é feita a troca com uma chave pública recebida (que será do outro agente com quem se está a comunicar) que irá gerar uma chave partilhada (a mesma nos dois agentes). Esta chave é ainda passada a uma função de derivação de chaves, para permitir uma mistura de informação que a torna mais segura.<br>
Para a chave poder ser gerada, é preciso saber a chave pública do outro agente, para isso, foi também criada uma função que devolve a chave pública gerada.

In [2]:
parameters = dh.generate_parameters(generator=2, key_size=1024)

def get_public_key(private_key):
        return private_key.public_key()

def derivate_key(private_key,public):
        shared_key = private_key.exchange(public)
        
        derived_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=b'handshake data',
        ).derive(shared_key)

        return derived_key

Depois da chave gerada, é o momento de enviar a mensagem para o agente Receiver.<br>
Para isso, é primeiro gerada uma assinatura, usando o HMAC que será concatenada com a mensagem cifrada e enviada ao Receiver para que este possa verificar o remetente.

In [3]:
def auth(message,derived_key):
        h = hmac.HMAC(derived_key, hashes.SHA256())
        h.update(message)
        return h.finalize()

De seguida, é usada a função *urandom()* do pacote *os* do *python* para gerar o *nonce* necessário para cifrar a mensagem.<br>
A cifra usada foi a AESGCM. Depois de tudo gerado e da mensagem cifrada, esta é concatenada à assinatura e ao *nonce* e enviada para o Receiver.

In [9]:
def send_message(message,derived_key):
        signature = auth(b'this is a message to check the signature',derived_key)
        message = message.encode('utf-8')
        nonce = os.urandom(16)
        aesgcm = AESGCM(derived_key)
        ct = aesgcm.encrypt(nonce, message, b'some associated data')

        return signature + nonce + ct

Do lado do Receiver, os dados são recebidos e desempacotados.

In [10]:
def unpack_data(dados):
      signature = dados[0:32]
      nonce = dados[32:32+16]
      ct = dados[32+16:]

      return signature, nonce, ct

Depois de ter os dados desempacotaos e separados, é necessário fazer a verificação da assinatira da mensagem recebida. Novamente, usando o HMAC.

In [11]:
def verify(signature,derived_key):
      h = hmac.HMAC(derived_key, hashes.SHA256())
      h.update(b'this is a message to check the signature')
      return h.verify(signature)

Se a assinatura for verificada, passa-se então à decifragem da mensagem pela cifra AESGCM.

In [14]:
def read_message(ct, derived_key):
      signature, nonce, ct = unpack_data(ct)
      try :
          # verifica se o digest gerado acima é igual ao digest recebido como parâmetro
          verify(signature,derived_key)
      except:
          raise Exception("Falha na autenticidade da chave") 

      aesgcm = AESGCM(derived_key)
      texto_limpo = aesgcm.decrypt(nonce, ct, b'some associated data')
    
      return texto_limpo.decode('utf-8')

Finalmente, enviamos a mensagem do Emitter para o Receiver.

In [17]:
def main():
    rv_private_key = parameters.generate_private_key()
    em_private_key = parameters.generate_private_key()
    em_derived_key = derivate_key(em_private_key,get_public_key(rv_private_key))
    rv_derived_key = derivate_key(rv_private_key,get_public_key(em_private_key))
    
    dados = send_message("Estruturas Criptográficas é uma unidade curricular do perfil de CSI", em_derived_key)
    print('encrypted text:',dados)

    try:
        pt = read_message(dados,rv_derived_key)

        print('decrypted text:', pt)
    except:
        print("Falha na autenticação da chave")  

main()

encrypted text: b'\xae\xd1H;Tr\xea\xd5\x05\xc7\x96\xbf}9\xa4\xceT\xdbO\x1b\tH\x84#\x8a\xf9\xdb\xffrl\xe4\x83\x8f\x94\xe5\x7f\x8d3\x19\x10\xa6\x92\\y\x85o\x9f\xa4\xe35\x87\xc3\xc2\xe6\x11s\x87.*\xfc\xfa\x87\x9e \xba\xee\xce\xf4|\xb3\xc9\xa0\x87O\xf0 k\x1eZ\xb4\x15+G\x98F.\xdd\n\xfb\xfb9VC\x90\x83\xf78\xb3\xa7W\xcd\x90F[\xac\xc7q\xc1\xd8[\x01\x1e\xb4R+\n.\x07\xb556\xa4D8\x1f+\x19\x03\x18\xbb\xec\xd1\xdf'
decrypted text: Estruturas Criptográficas é uma unidade curricular do perfil de CSI


# Exercicio 2
## Criar uma cifra com autenticação de meta-dados a partir de um PRG


Como objetivo inicial, esperava-se que o nosso grupo implementasse um gerador pseudo-aleatório do usando uma função XOF("extended output function") - SHAKE256 - de forma a gerar uma sequência de palavras, cada uma com 64 bits.


Para começar, criamos uma função que recebe a password e a partir de uma *KDF* - no nosso caso decidimos escolher a **PBKDF2HMAC**, mas também poderíamos ter usado a *HKD*. Esta decisão foi tida em conta com a leitura da documentação da lib *Crypthography*, a qual inferia que há diferentes tipos de funções de derivação para diferentes tipos de propósitos. Como o nosso objetivo seria criar uma seed para o gerador, escolhemos um dos que era recomendado. 

In [ ]:
def derive_key(password):
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        )

    return kdf.derive(password)

De seguida, e conforme enunciado, era necessário criar o gerador, que tivesse um limite de palavras definido por um valor inicialmente parametrizado como **N**. Com a seed gerada pela *key derivation function* que escolhemos seria possível gerar 2^N palavras de 64 bits cada.
Seguindo o requisitado para este projeto, era necessário usar o **SHAKE256** para definir o gerador - como se sabe o *SHAKE256* é uma *extendable-output function* e define-se o seu uso tendo em conta a necessidade de autenticação do criptograma e geração de palavras usando uma XOF. 


In [ ]:
def prg(seed,N):
    digest = hashes.Hash(hashes.SHAKE256(8 * pow(2,N))) # sequencia palavras 64 bits / 8 = 8 bytes
    digest.update(seed)
    msg = digest.finalize()
    return msg

A partir da sequência de bits representativa da lista de palavras, gerada pelo PRG, conseguimos aplicar a cifra **OTP**.
Assumindo a variante construída em 1919, é necessário definir a operação *XOR* em contraste com a sua versão original, criada com adição modular. É então preciso aplicar esta operação ao processo de cifragem e decifragem, sendo que, a partir de duas sequências de bytes é aplicado o XOR a ambas.


In [ ]:
BLOCK = 8 # bytes; representativo de cada bloco que define uma palavra (64 bits)

In [ ]:
def pad_divide(message):
    x = []
    for i in range (0,len(message), BLOCK):
        next = i+BLOCK
        x.append(message[i:next])
    return x

É importante denotar o mecanismo associado a estes processos de cifragem e decifragem. Inicialmente é feito o *padding*, de forma a que os blocos de mensagens sejam garantidamente múltiplos. Após isto, e para simplicidade prática,é necessário dividir a mensagem em blocos de 8 bytes, cada um representando uma palavra.

In [ ]:
def cipher(k,msg):
    ciphertext = b''
    pad = padding.PKCS7(64).padder()
    
    padded = pad.update(msg) + pad.finalize()
    p = pad_divide(padded)

    for x in range (len(p)): # Percorre blocos do texto limpo
        for bloco, byte in enumerate(p[x]): # Percorre bytes do bloco do texto limpo
            ciphertext += bytes([byte ^ k[x:(x+1)*BLOCK][bloco]]) # xor of 2 bit sequences plain text and cipher_key
    return ciphertext


In [ ]:

def decipher(k,ciphertext):
    plaintext=b''
    
    p=pad_divide(ciphertext)

    for x in range (len(p)): # Percorre blocos do texto cifrado
        for bloco, byte in enumerate(p[x]): # Percorre bytes do bloco do texto cifrado
            plaintext += bytes([byte ^ k[x:(x+1)*BLOCK][bloco]]) 
    
    # Algoritmo para retirar padding para decifragem
    unpadder = padding.PKCS7(64).unpadder()
    # Retira bytes adicionados 
    unpadded = unpadder.update(plaintext) + unpadder.finalize()
    return unpadded.decode("utf-8")

In [ ]:
    
cipher_key = derive_key(b'password')
msg = prg(cipher_key,2)
mensagem = b"Ultra secret message"

ct = cipher(msg,mensagem)
dt = decipher(msg,ct)


print("OG TEXT: ", mensagem)
print("CT:  ", ct)
print("DT:  ", dt)